In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
import pandas as pd

file_path = '/content/drive/MyDrive/filtered_NOTEEVENTS.csv'  # update this if needed
df = pd.read_csv(file_path)

# Preview the data
df.head()


,ROW_ID,SUBJECT_ID,HADM_ID,CHARTDATE,CHARTTIME,STORETIME,CATEGORY,DESCRIPTION,CGID,ISERROR,TEXT,length,clean_text
0,42102,58526,100001.0,2117-09-17,NaN,NaN,Discharge summary,Report,NaN,NaN,Admission Date: [**2117-9-11**] ...,8285,admission date: discharge date: date of birth:...
1,19215,54610,100003.0,2150-04-21,NaN,NaN,Discharge summary,Report,NaN,NaN,Admission Date: [**2150-4-17**] ...,9981,admission date: discharge date: date of birth:...
2,8772,9895,100006.0,2108-04-17,NaN,NaN,Discharge summary,Report,NaN,NaN,Admission Date: [**2108-4-6**] Discharg...,5943,admission date: discharge date: date of birth:...
3,50238,23018,100007.0,2145-04-07,NaN,NaN,Discharge summary,Report,NaN,NaN,Admission Date: [**2145-3-31**] ...,7857,admission date: discharge date: date of birth:...
4,21119,533,100009.0,2162-05-21,NaN,NaN,Discharge summary,Report,NaN,NaN,Admission Date: [**2162-5-16**] ...,11276,admission date: discharge date: date of birth:...


In [4]:
from textwrap import wrap

chunks = []
max_chunk_size = 1000  # character-based chunks (safe for LLMs)

for note in df["TEXT"]:
    if pd.notna(note):
        cleaned = note.replace("\n", " ").strip()
        note_chunks = wrap(cleaned, width=max_chunk_size)
        chunks.extend(note_chunks)


In [5]:
import pickle

with open('/content/drive/MyDrive/chunks.pkl', 'wb') as f:
    pickle.dump(chunks, f)

print(f"✅ Saved {len(chunks)} chunks!")

✅ Saved 580083 chunks!


In [6]:
!pip install chromadb langchain

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.3/67.3 kB 6.4 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 18.3/18.3 MB 40.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.4/2.4 MB 73.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 94.9/94.9 kB 7.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 284.2/284.2 kB 20.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 68.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 101.6/101.6 kB 9.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 16.0/16.0 MB 74.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 55.9/55.9 kB 4.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 188.4/188.4 kB 15.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 65.3/65.3 kB 6.4 MB/s eta 0:00:

In [7]:
import pandas as pd

embeddings_df = pd.read_csv('/content/drive/MyDrive/clinical_note_embeddings.csv')
embeddings = embeddings_df.to_numpy().tolist()

print(f"✅ Loaded {len(embeddings)} embeddings with {len(embeddings[0])} dimensions each")

✅ Loaded 52726 embeddings with 769 dimensions each


In [8]:
pip install -U langchain-community

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 43.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.9/50.9 kB 3.9 MB/s eta 0:00:00


In [9]:
!pip install faiss-cpu langchain


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 30.7/30.7 MB 12.7 MB/s eta 0:00:00


In [10]:
# Trim chunks to match number of embeddings
chunks = chunks[:len(embeddings)]
print(f"🧠 Matched chunks: {len(chunks)}, embeddings: {len(embeddings)}")


🧠 Matched chunks: 52726, embeddings: 52726


In [11]:
from langchain.docstore.document import Document

documents = [
    Document(page_content=chunks[i], metadata={"source": f"chunk_{i}"})
    for i in range(len(chunks))
]


In [12]:
from langchain.embeddings.base import Embeddings
import numpy as np

class PrecomputedEmbeddings(Embeddings):
    def __init__(self, vectors):
        self.vectors = vectors

    def embed_documents(self, texts):
        return self.vectors

    def embed_query(self, text):
        raise NotImplementedError("Only storing vectors for now")


In [13]:
from langchain.vectorstores import FAISS

embedding_vectors = np.array(embeddings)
embedder = PrecomputedEmbeddings(vectors=embedding_vectors)

faiss_db = FAISS.from_documents(
    documents=documents,
    embedding=embedder
)

# Save locally
faiss_db.save_local("/content/faiss_index")
print("✅ FAISS vector store saved locally.")


✅ FAISS vector store saved locally.


In [14]:
import shutil

shutil.copytree("/content/faiss_index", "/content/drive/MyDrive/MedRAG/faiss_index")
print("✅ FAISS DB backed up to Drive.")


FileExistsError: [Errno 17] File exists: '/content/drive/MyDrive/MedRAG/faiss_index'

In [ ]:
!pip install faiss-cpu langchain transformers sentence-transformers


In [ ]:
from langchain.vectorstores import FAISS
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.llms import HuggingFacePipeline
from langchain.chains import RetrievalQA
from transformers import pipeline


In [ ]:
from langchain.embeddings import HuggingFaceEmbeddings

embedding_model = HuggingFaceEmbeddings(model_name="emilyalsentzer/Bio_ClinicalBERT")


In [ ]:
from langchain.vectorstores import FAISS

faiss_db = FAISS.load_local(
    folder_path="/content/drive/MyDrive/MedRAG/faiss_index",
    embeddings=embedding_model,
    index_name="index",
    allow_dangerous_deserialization=True
)

retriever = faiss_db.as_retriever(search_kwargs={"k": 5})


In [ ]:
# ✅ Load FREE HuggingFace LLM: google/flan-t5-base
hf_pipeline = pipeline(
    task="text2text-generation",
    model="google/flan-t5-base",
    max_length=512
)

llm = HuggingFacePipeline(pipeline=hf_pipeline)


In [ ]:
# ✅ Create the RAG chain
rag_chain = RetrievalQA.from_chain_type(
    llm=llm,
    retriever=retriever,
    chain_type="stuff"
)


In [ ]:
# ✅ (Optional) Ask a test question before UI
query = "Summarize the discharge instructions of the patient."
response = rag_chain.run(query)
print("🧠 Response:", response)

In [ ]:
import numpy as np

vec = embedding_model.embed_query("test")
print("Embedding dimension:", np.array(vec).shape)
